In [2]:
import dash 
from dash import dcc, html, Input, Output
import pandas as pd 
import plotly.graph_objects as go 
import sys 

sys.path.append('../')

import omdutils as omd

In [3]:
app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Input(id='my-id', value='initial value', type='text'),
    html.Div(id='my-div', style={'border':'2px blue solid'})
])

@app.callback(Output(component_id='my-div', component_property='children'),
              Input(component_id='my-id', component_property='value'))

def get_message(input_value):
    return f'Input value was: "{input_value}"'

app.run(debug=True)

In [4]:
# df = pd.read_csv('../Data/gapminderDataFiveYear.csv')
# db = '../SourceData/datasets.db'
# omd.to_sqlite(df, 'gapminderDataFiveYear', db)
db = '../SourceData/datasets.db'
sql = 'SELECT * FROM gapminderDataFiveYear'
df = omd.from_sqlite(sql, db)
df.head()

,country,year,pop,continent,lifeExp,gdpPercap
0,Afghanistan,1952,8425333.0,Asia,28.801,779.445314
1,Afghanistan,1957,9240934.0,Asia,30.332,820.853030
2,Afghanistan,1962,10267083.0,Asia,31.997,853.100710
3,Afghanistan,1967,11537966.0,Asia,34.020,836.197138
4,Afghanistan,1972,13079460.0,Asia,36.088,739.981106


In [5]:
continent_colors = {
    'Asia': 'red',
    'Europe': 'blue',
    'Africa': 'green',
    'Americas': 'orange',
    'Oceania': 'purple'
}

app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Dropdown(id='year-picker', 
                 options=[{'label': i, 'value': i} for i in df['year'].unique()],
                 value=df['year'].min()),
    dcc.Graph(id='my-graph')
])

@app.callback(Output('my-graph', 'figure'),
              Input('year-picker', 'value'))
def update_plot(selected_year):
    filtered_df = df[df['year'] == selected_year]
    data = []
    for continent, color in continent_colors.items():
        continent_df = filtered_df[filtered_df['continent'] == continent]
        data.append(go.Scatter(
            x=continent_df['gdpPercap'],
            y=continent_df['lifeExp'],
            text=continent_df['country'],
            mode='markers',
            marker=dict(
                size=15,
                opacity=0.6,
                line=dict(width=0.5, color='white'),
                color=color
            ),
            name=continent  # Name for the legend
        ))
    layout = go.Layout(
        title='Life Expectancy vs GDP Per Capita',
        xaxis=dict(title='GDP Per Capita', type='log'),
        yaxis=dict(title='Life Expectancy'),
        hovermode='closest'
    )
    return {'data': data, 'layout': layout}

app.run(debug=True)